In [ ]:
import pandas as pd
import glob
from autogluon.tabular import TabularPredictor

combined_df = pd.read_feather("datasets/predict_yolomodel.feather")
label_column = "yolomodel"

excluded_features = []
importance_dfs = []
summaries = []

# Number of iterations for training with excluded features
iterations = 13

for i in range(iterations):
    # Remove excluded features from the dataset
    current_df = combined_df.drop(columns=excluded_features, errors="ignore")

    # Train AutoGluon TabularPredictor with a time limit
    time_limit_seconds = 10  # example time limit: 60 seconds
    predictor = TabularPredictor(label=label_column).fit(current_df, time_limit=time_limit_seconds, verbosity=1,)

    # Get the best model and feature importance
    best_model = predictor.fit_summary()
    summaries.append(best_model)
    importance_df = predictor.feature_importance(current_df)

    # Get the most important column and add it to excluded features
    most_important_feature = importance_df.index[0]
    importance_dfs.append(importance_df.copy())
    excluded_features.append(most_important_feature)

    print(f"Iteration {i+1}: Excluded Feature - {most_important_feature}")
    print("Currently Excluded Features:", excluded_features)
    print("Most Important Columns in this iteration:", importance_df.index.tolist())

In [ ]:
for model in summaries:
    print(model["leaderboard"]["score_val"][0])

In [ ]:
for importance_df in importance_dfs:
    print(importance_df)

In [ ]:
current_df[label_column].unique()

In [ ]:
import seaborn as sns
combined_df['{"__name__": "instance_device:node_disk_io_time_weighted_seconds:rate5m", "instance": "worker"}'].plot()

In [ ]:
import seaborn as sns

# Filter the dataset for a subset of 'yolomodels'
subset_yolomodels = ["yolo11n", "yolo11s", "yolo11m"]  # Example subset
filtered_df = combined_df[combined_df['yolomodel'].isin(subset_yolomodels)]

# Compute the average for each 'yolomodel'
average_df = combined_df.groupby('yolomodel', as_index=False).mean()

# Plot the average for each 'yolomodel'
sns.barplot(data=average_df, x='yolomodel', y='{"__name__": "instance_device:node_disk_io_time_weighted_seconds:rate5m", "instance": "worker"}')

In [ ]:
filtered_df[('{"__name__": "instance_device:node_disk_io_time_weighted_seconds:rate5m", "instance": "worker"}')]

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

# Filter the dataset for a subset of 'yolomodels'
subset_yolomodels = ["yolo11n", "yolo11s", "yolo11m"]  # Example subset
filtered_df = combined_df[combined_df['yolomodel'].isin(subset_yolomodels)]

# Compute the average for each 'yolomodel'
average_df = combined_df.groupby('yolomodel', as_index=False).mean()

# Plot the average for each 'yolomodel'
for feature in excluded_features:
    sns.barplot(data=average_df, x='yolomodel', y=feature)
    plt.title(feature)
    plt.show()